In [1]:
%load_ext autoreload
%autoreload 2

import sys, os

# Go up one directory from `b/` to project root
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(root_dir)

In [2]:
import logging
from neurosurfer.models.chat_models.transformers import TransformersModel
from neurosurfer.models.chat_models.unsloth import UnslothModel
from neurosurfer import config 
import torch
torch.cuda.empty_cache()

# falcon-7b-instruct
# "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
# "model_name": "/home/nomi/workspace/Model_Weights/falcon-7b-instruct",

DEFAULT_TRANSFORMERS_MODEL_PARAMS = dict({
    "model_name": "/home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit",
    "max_seq_length": 4096,
    "load_in_4bit": True,
    "enable_thinking": False,
    "reasoning_effort": "low",
    "add_special_tokens": False,
    "verbose": False
})

LOGGER = logging.getLogger()
LLM = TransformersModel(
    **DEFAULT_TRANSFORMERS_MODEL_PARAMS,
    stop_words=["Observation:"],
    logger = logging.getLogger(),
)

╔══════════════════════════════════════════════════════════════════╗
║                                                                  ║
║ ▓▓▓▓▓   ▓▓▓▓                                  ▓▓▓                ║
║  ▓▓ ▓▓   ▓▓  ▓▓▓▓ ▓  ▓ ▓ ▓ ▓▓▓▓ ▓▓▓ ▓  ▓ ▓ ▓  ▓   ▓▓▓▓ ▓ ▓       ║
║  ▓▓  ▓▓  ▓▓  ▓▁▁▓ ▓  ▓ ▓▓▏ ▓  ▓ ▓▁  ▓  ▓ ▓▓▏ ▓▓▓  ▓▁▁▓ ▓▓        ║
║  ▓▓   ▓▓ ▓▓  ▓    ▓  ▓ ▓   ▓  ▓   ▓ ▓  ▓ ▓    ▓   ▓    ▓         ║
║ ▓▓▓▓   ▓▓▓▓▓ ▓▓▓▓ ▓▓▓▓ ▓   ▓▓▓▓ ▓▓▓ ▓▓▓▓ ▓    ▓   ▓▓▓▓ ▓         ║
║ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ ║
║ Orchestrate Agents - RAG - SQL Tools - Multi-LLM - FastAPI Ready ║
║ Faster builds, clearer flows, production-first                   ║
║                                                                  ║
║ Version: 0.1.0 | Python: 3.12.12                                 ║
║ OS: Linux 6.17.0-7-generic (x86_64)                              ║
║ Torch: 2.8.0   CUDA: yes (12.8)                                  ║
║ MPS: no (built: False)          

/home/nomi/micromamba/envs/LLMs/lib/python3.12/importlib/__init__.py:90: UserWarning: WARNING: Unsloth should be imported before transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  return _bootstrap._gcd_import(name[level:], package, level)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
#### Unsloth: `hf_xet==1.1.10` and `ipykernel>6.30.1` breaks progress bars. Disabling for now in XET.
#### Unsloth: To re-enable progress bars, please downgrade to `ipykernel==6.30.1` or wait for a fix to
https://github.com/huggingface/xet-core/issues/526
Switching to PyTorch attention since your Xformers is broken.

Unsloth: Xformers does not work in RTX 50X, Blackwell GPUs as of yet. Please build from source via
```
pip install ninja
pip install -v --no-build-isolation -U git+https://github.com/facebookresearch/xformers.git@main#egg=xformers
```

🦥 Unsloth Zoo will now patch everything to make training faster!
INFO     | 2025-12-04 23:23:59 | transformers.py:init_model | Initializing Transformers model.
WARNING  | 2025-12-04 23:23:59 | transformers.py:init_model | Model is already quantized. Ignoring load_in_4bit=True.


`torch_dtype` is deprecated! Use `dtype` instead!


INFO     | 2025-12-04 23:24:00 | modeling.py:get_balanced_memory | We will use 90% of the memory on device 0 for storing the model, and 10% for the buffer to avoid OOM. You can set `max_memory` in to a higher value to use more memory (at your own risk).


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO     | 2025-12-04 23:24:04 | transformers.py:init_model | Transformers model initialized successfully.


In [4]:
# streaming response example
from IPython.display import display, Markdown, clear_output

system_prompt = """You are a joke teller"""
user_prompt = """Tell me a short and light-hearted joke."""

stream_response = LLM.ask(
    system_prompt=system_prompt,
    user_prompt=user_prompt,
    stream=True
)

# print(stream_response.choices[0].message.content)
md_display = display(Markdown(""), display_id=True)
for chunk in stream_response:
    chunk = chunk.choices[0].delta.content or ""
    print(chunk, flush=True, end="")

Why don't skeletons fight each other?  
Because they don’t have the *guts*! 😄

## Agent Test

**Normal Response**

In [5]:
# agent normal response
from neurosurfer.agents import Agent, AgentConfig
# from neurosurfer.tracing import RichTracer
from pydantic import BaseModel

agent_config = AgentConfig(
    strict_tool_call=True,
    return_stream_by_default=True,
    temperature=0.7,
    max_new_tokens=4096,
)
agent = Agent(llm=LLM, config=agent_config, log_traces=True)

# normal response
print("Normal Response:")
agent_response = agent.run(user_prompt="What is AI (one line)?", stream=False)
# print(agent_response.response)

# # streaming response
# print("\n\nStreaming Response:")
# for c in agent.run(user_prompt="What are top 3 applications of AI (one line)?").response:
#     print(c, flush=True, end="")


Normal Response:
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.free_text_call'
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.free_text_call' took 0.698s; error=False
 ◀ [1][step.agent] agent_id='main_agent' label='agent.run' took 0.701s; error=False
[main_agent] Tracing End!

Final response:
AI, or Artificial Intelligence, is the simulation of human intelligence in machines that are programmed to think, learn, and perform tasks typically requiring human cognition.


**Structured Response**

In [6]:
# Structured Response examples
class AIApplication(BaseModel):
    title: str
    description: str

class AI(BaseModel):
    definition: str
    history: str
    modern_frameworks: str
    applications: list[AIApplication]

user_query = "What is AI and list 3 of its top application, and 3 concerns."
agent_response = agent.run(user_prompt=user_query, output_schema=AI)

print("\n\nStructured Response:")
print(agent_response.response.json_obj)  

WARNING  | 2025-12-04 23:24:20 | agent.py:run     | `output_schema` provided with `stream=True`; forcing non-streaming structured output.
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.structured_call.first_pass'
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.structured_call.first_pass' took 4.148s; error=False
 ◀ [1][step.agent] agent_id='main_agent' label='agent.run' took 4.161s; error=False
[main_agent] Tracing End!



Structured Response:
{
  "definition": "Artificial Intelligence (AI) refers to the simulation of human intelligence in machines that are programmed to think, learn, and make decisions.",
  "history": "The concept of AI dates back to the 1950s, with the term 'artificial intelligence' coined in 1956. Early research focused on creating machines that could perform tasks requiring human intelligence, such as problem-solving and language understa

In [7]:
agent_response.traces.steps[1].model_dump()

{'step_id': 2,
 'kind': 'llm.call',
 'label': 'agent.structured_call.first_pass',
 'node_id': None,
 'agent_id': 'main_agent',
 'started_at': 1764876260.2168481,
 'duration_ms': 4148,
 'inputs': {'schema': 'AI',
  'system_prompt_len': 52,
  'user_prompt_len': 61,
  'user_prompt': 'What is AI and list 3 of its top application, and 3 concerns.',
  'system_prompt': "You are a precise and rule-abiding assistant.  \nYour task is to produce only a single valid JSON object following the schema below.\n\nStructured Output Contract:\n- Output only JSON — no markdown, code fences, or explanations.  \n- JSON must be strictly valid (RFC 8259): use double quotes for all keys and string values.  \n- Do not include extra keys or any text outside the JSON object.  \n- All required fields must be present, even if empty.  \n- Arrays must contain at least one object when applicable.  \n- The JSON must be a single complete object (not pretty-printed, no trailing commas).  \n- Failure to comply with this s

**Tool Calling Test**

In [9]:
from neurosurfer.agents.agent import Agent
from neurosurfer.tools.toolkit import Toolkit
from neurosurfer.tools.code_execution.python_exec_tool import PythonExecTool


py_exec_tool = PythonExecTool(llm=LLM)
toolkit = Toolkit(tools=[py_exec_tool])

# print("Tool description:")
# print(py_exec_tool.get_tool_description())
# print()

agent = Agent(llm=LLM, toolkit=toolkit, log_traces=True)

print("Agent with choice between tools and plain text:")
user_prompt = """Using the Python tool, compute the determinant of this 50x50 matrix whose entries follow the rule A[i][j] = (i*j) % 7 + sin(i). Return only the final numeric determinant"""
agent_response = agent.run(user_prompt=user_prompt, strict_tool_call=True, stream=False)

# for chunk in agent_response.response.returns:
#     pass

print(agent_response.response)

# print("\n\nAgent with strict tool call:")
# agent_response = agent.run(user_prompt="What is one forth of a 100?", strict_tool_call=True)
# print(agent_response.response)


# for chunk in agent_response.response.returns:
#     print(chunk, end="", flush=True)


INFO     | 2025-12-03 09:36:07 | toolkit.py:register_tool | Registered tool: python_execute
Agent with choice between tools and plain text:
🧠 Thinking...

[main_agent] Tracing Start!
 ▶ [1][step.agent] agent_id='main_agent' label='agent.run'
     ▶ [2][step.llm.call] agent_id='main_agent' label='agent.route_and_call.router_llm_call'
        INFO: Selected tool: python_execute
        INFO: Raw inputs: {'task': 'Compute the determinant of a 50x50 matrix where each entry A is given by (i*j) % 7 + sin(i).'}
     ◀ [2][step.llm.call] agent_id='main_agent' label='agent.route_and_call.router_llm_call' took 1.406s; error=False
     ▶ [3][step.tool.execute] agent_id='main_agent' label='agent.route_and_call.tool_execute'
        INFO: Tool 'python_execute' Tool Return: Here are the results from executing Python code for your task:
        INFO: 
        INFO: 0.0
        INFO: 
        INFO: Python code used:
        INFO: ```python
        INFO: imp...
     ◀ [3][step.tool.execute] agent_id='m

In [23]:
# print(agent_response.response.extras['python_last_code']['value'])
print(agent_response.response.extras['python_last_result_summary'])

{'value': {'kind': 'float64', 'value': np.float64(0.0)}, 'description': 'Primitive result from the last python_execute call.', 'visible_to_llm': True}


**Let's solve this problem using CodeAgent**

In [15]:
# !pip install tabulate

In [10]:
from neurosurfer.agents.code.agent import CodeAgent

code_agent = CodeAgent(llm=LLM, log_traces=True)

files_context = {
    "Student Degree College Data.csv": {
        "path": "/home/nomi/Downloads/archive/Student Degree College Data.csv",
        "mime": "text/csv",
        "size": 422368,
    }
}

# code_agent.set_persistent_memory(files_context=files_context)
# mem_injected = code_agent.memory.resolve_keys(['files_context'])
# print(mem_injected)

# query = "Using the uploaded 'Student Degree College Data.csv', get me a report explaining the data in very details."
# query = "Use the python tool to generate a line plot of y = x^3 − 4x + 1 for x in the range −10 to 10. Return the image."
# query = "Using python, generate an Excel file with two sheets: (1) primes up to 5000 and (2) Fibonacci numbers up to 100,000. Provide the file for download."
query = "First, analyze the uploaded 'Student Degree College Data.csv' file. Based on the data, ask yourself a question that must make use of the Python tool."
resp = code_agent.run(
    query=query,
    files_context=files_context,
    workdir=None,    # take current working directory
    post_process="none",   # optional
    stream=False,
)

# print(resp.response)


INFO     | 2025-12-03 09:40:15 | toolkit.py:register_tool | Registered tool: python_execute
🧠 Thinking...

[CodeAgent] Tracing Start!
 ▶ [1][step.react_agent] agent_id='CodeAgent' label='react_agent.run'
     ▶ [2][step.llm.call] agent_id='CodeAgent' label='react_agent.loop.reason_llm_call'
        Thought: I need to inspect the 'Student Degree College Data.csv' file to understand its structure and contents before performing any computations. I will start by loading the data and displaying the first few rows to identify the column names and data types.
        
        Action: {
          "tool": "python_execute",
          "inputs": {
            "task": "Load the 'Student Degree College Data.csv' file and display the first few rows to inspect the data structure."
          },
          "memory_keys": [],
          "final_answer": false
        }        
        [🔧] Tool Selected: python_execute
         ▶ [3][step.tool.execute] agent_id='CodeAgent' label='react_agent.loop.tool_execut

In [29]:
# code_agent.memory.get_persistent(key="files_context")
# code_agent.memory.llm_visible_summary()

# code_agent.memory.get_ephemeral(key="python_last_result_summary")

# print(resp.traces.steps[4].outputs['memory_update']['python_last_code']['value'])

In [21]:
resp.traces.steps[4].model_dump()

# print()

# print(resp.traces.steps[4].outputs['tool_return'])
print(resp.traces.steps[4].outputs['memory_update']['python_last_code']['value'])

# resp.traces.model_dump()


import pandas as pd

# Load the CSV file
file_path = files["Student Degree College Data.csv"]["path"]
df = pd.read_csv(file_path)

# Calculate the average marks for first and second choice subjects
average_first_choice = df['first_choice_marks'].mean()
average_second_choice = df['second_choice_marks'].mean()

result = {
    "average_first_choice_marks": average_first_choice,
    "average_second_choice_marks": average_second_choice
}


In [ ]:
from typing import List
from neurosurfer.server.db.db import SessionLocal
from neurosurfer.server.db.models import NSFile
import re

ALLOWED_NAME = re.compile(r"[^a-zA-Z0-9._-]+")

def _collection_for(user_id: int, thread_id: int) -> str:
    return _safe_collection_name(f"ns_vdb_u{user_id}_t{thread_id}")

def _safe_collection_name(s: str) -> str:
    s = ALLOWED_NAME.sub("_", s).strip("._-")
    return s or "ns_vdb_default"


user_id = 1
thread_id = 21
message_id = 1
collection_name = _collection_for(user_id, thread_id)

db = SessionLocal()
files = (
    db.query(NSFile)
    .filter(
        NSFile.user_id == user_id,
        NSFile.thread_id == thread_id,
        NSFile.collection == collection_name,
        NSFile.ingested == True
    )
    .all()
)
files_block_lines: List[str] = []
for f in files:
    flag = "yes" if message_id and f.message_id == message_id else "no"
    files_block_lines.append(
        f"Filename: {f.filename}\n"
        f"Attached_to_current_message: {flag}\n"
        f"Summary:\n{f.summary or '(no summary)'}\n"
    )
files_block = "\n\n".join(files_block_lines)

print(files_block)




Filename: Student Degree College Data.csv
Attached_to_current_message: no
Summary:
This file is about a dataset containing information on students' academic records, including their personal details, high school performance, and choices of subjects and streams. The data includes fields such as form number, name, category, gender, total marks, passing year, and subject preferences. 

Keywords: student records, academic performance, subject choices, high school data, form number, category


Filename: rag_agent.md
Attached_to_current_message: no
Summary:
This file is about the RAGRetrieverAgent module, which provides a modular retrieval system for Retrieval-Augmented Generation. It supports various vector stores, embedders, and LLMs, and includes tools for retrieving, formatting, and trimming context to fit model token limits. The module also offers utilities for file picking and best practices for efficient retrieval and generation.

Keywords: RAGRetrieverAgent, Retrieval-Augmented Gener

## SERVER SIDE

In [39]:
from neurosurfer.server.app import NeurosurferApp
from neurosurfer.config import config

# Create app instance
ns = NeurosurferApp(
    app_name="Neurosurfer API",
    api_keys=[],
    enable_docs=True,
    cors_origins=["*"],
    host="0.0.0.0",
    port=8000,
    reload=config.app.reload,
    log_level=config.app.logs_level,
    workers=config.app.workers
)

# Initialize models
ns.register_model(
    model=LLM,
    family="llama",
    provider="Unsloth",
    description="Proxy to Llama"
)

# Initialize RAG
ns._init_rag(embedder=ns._embedder, llm=ns._llm)

# Initialize agent
ns._init_agent()

INFO     | 2025-12-05 00:04:50 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-12-05 00:04:50 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-12-05 00:04:50 | app.py:__init__  | Default embedder: intfloat/e5-large-v2 successfully initialized. Replace it by calling app.register_model(model: BaseEmbedder)
INFO     | 2025-12-05 00:04:50 | db.py:init_db    | Database initialized successfully in sqlite:////home/nomi/.cache/Neurosurfer/app-storage/app.db
INFO     | 2025-12-05 00:04:50 | app.py:register_model | LLM /home/nomi/workspace/Model_Weights/Qwen3-8B-unsloth-bnb-4bit successfully registered.
WARNING  | 2025-12-05 00:04:50 | agent.py:__init__ | No vectorstore provided to RAGAgent, using default ChromaVectorStore. Initializing default collection `neurosurfer-rag-agent`
[Init] ChromaVectorStore initialized with collection: neurosurfer-rag-agent
INFO     | 2025-12-05 00:04:50 | orchestrator.py:__init

In [37]:
# test the server
# Prepare inputs

user_query = "Check what this file is about? Give me detailed answer, not just summary."
# Minimal chat history excluding system messages, max 10
# num_recent = 10
# conversation_messages = args.messages.converstaion
# chat_history = conversation_messages[-num_recent:-1]

# Model call (stream or non-stream handled by router)
# kwargs = {"temperature": args.temperature, "max_new_tokens": args.max_tokens}

user_id = 1
thread_id = 1
message_id = 1
has_files_message = False

agent_return = ns.run_agent(
    user_id=user_id,
    thread_id=thread_id,
    message_id=message_id,
    user_query=user_query,
    has_files_message=has_files_message,
)

for chunk in agent_return.response:
    ...
    # chunk = chunk.choices[0].delta.content or ""
    # print(chunk, end="", flush=True)

Files Context:
{'agents_index.md': {'path': '/home/nomi/.cache/Neurosurfer/app-storage/ns_users_1/ns_threads_1_1/files/file_f8e989535617405e8ce5d6a12daaa97d.md', 'mime': 'text/markdown', 'size': 5956}}

User Prompt for Agent:
# User query:
Check what this file is about? Give me detailed answer, not just summary.

# Uploaded files for this thread (with summaries):
Filename: agents_index.md
Attached_to_current_message: no
Summary:
This file is about AI agents that can reason, use tools, and solve complex tasks, with a focus on three specific agents: ReActAgent for general reasoning, SQLAgent for database analytics, and RAGRetrieverAgent for document Q&A. It explains their purposes, use cases, and provides code examples for quick start.

Keywords: AI agents, ReActAgent, SQLAgent, RAGRetrieverAgent, tool routing, document Q&A



🧠 Thinking...

[MainChatAgent] Tracing Start!
 ▶ [1][step.react_agent] agent_id='MainChatAgent' label='react_agent.run'
     ▶ [2][step.llm.call] agent_id='MainCha

AttributeError: 'ReActConfig' object has no attribute 'verbose'

In [ ]:
# print(ns._agent.toolkit.get_tools_description())

Available tools:
Tool Name: `code_agent_run`
Description: Use the CodeAgent to answer questions that require Python-based analysis or computation over uploaded files (e.g. CSV, text, etc.). The CodeAgent can inspect file structure, compute statistics, create plots, and return summarized results.
When to use: Use this tool whenever the query clearly needs precise computation or data manipulation that is hard to do in your head, especially when the user has uploaded structured data (CSV, tables, logs). Examples: counting rows, aggregating metrics, computing correlations, or generating plots from uploaded files. Do NOT use this tool for general Q&A that does not depend on code or files.
Tool Inputs:
- `query`: string (required) — Natural-language description of what the CodeAgent should do. You can mention uploaded files by name and describe the analysis or computation needed.
- `files_hint`: array (optional) — Optional list of filenames that are especially relevant. If omitted, the runti

In [70]:
# agent_return.traces.model_dump()
# print(agent_return.traces.steps[0].inputs['scratchpad'])

from neurosurfer.agents.react import ToolCall

# tool_calls = [ToolCall(tool='python_execute', inputs={'task': "Load the 'Student Degree College Data.csv' file and display the first few rows to inspect the structure."}, final_answer=False, memory_keys=[], rationale=None, output='Here are the results from executing Python code for your task:\n\n|   form_no | name            | category   | minority   | gender   |   hs_total |   hs_pass_year | first_choice_sub   |   first_choice_marks | second_choice_sub   |   second_choice_marks | first_choice_stream   | second_choice_stream   |\n|----------:|:----------------|:-----------|:-----------|:---------|-----------:|---------------:|:-------------------|---------------------:|:--------------------|----------------------:|:----------------------|:-----------------------|\n|   1801371 | Susmita Bhowmik | UR         | NO         | F        |        381 |           2018 | Mathematics        |                   80 | Mathematics         |                    80 | Physical Science      | Physical Science       |\n|   1803096 | Ankur Das       | SC         | NO         | M        |        262 |           2018 | Education          |                   61 | Bengali             |                    58 | Arts                  | Arts                   |\n|   1801287 | Rashmita Tanti  | UR         | NO         | M        |        278 |           2018 | Human Physiology   |                   74 | Botany              |                    74 | Bio Science           | Bio Science            |\n|   1803173 | Sanjib Ghosh    | UR         | NO         | M        |        194 |           2018 | nan                |                    0 | nan                 |                     0 | Arts                  | Arts                   |\n|   1801077 | Swarupa Pal     | UR         | NO         | F        |        320 |           2015 | nan                |                    0 | nan                 |                     0 | Bio Science           | Bio Science            |'), ToolCall(tool='python_execute', inputs={'task': "Find the grades of Paheli Bhaumik from the 'Student Degree College Data.csv' file."}, final_answer=False, memory_keys=['python_last_result_summary'], rationale=None, output='Here are the results from executing Python code for your task:\n\n[\n  93,\n  93\n]')]
# tool_calls[1].__dict__
    
# print(agent_return.tool_calls[0].__dict__['output'])

import json
with open('agent_return.json', 'w') as f:
    json.dump(agent_return.traces.model_dump(), f)
# agent_return.traces.model_dump()

## ReAct Agent

In [19]:
import json 

# CityInfoTool
class CityInfoTool(BaseTool):
    """
    Simple read-only city info DB so the ReAct agent has something non-trivial to reason about.
    """
    spec = ToolSpec(
        name="city_info",
        description="Look up basic information about a city (population and timezone).",
        when_to_use="Use this tool when you need factual info about a city such as population or timezone.",
        inputs=[
            ToolParam(name="city", type="string", description="City name, e.g. 'Paris', 'Tokyo'.", required=True),
        ],
        returns=ToolReturn(type="string", description="A short JSON-formatted string with fields like population_millions and timezone."),
    )

    def __init__(self):
        # tiny in-memory DB
        self._db = {
            "paris": {
                "population_millions": 2.1,
                "timezone": "Europe/Paris",
            },
            "tokyo": {
                "population_millions": 13.9,
                "timezone": "Asia/Tokyo",
            },
            "dubai": {
                "population_millions": 3.6,
                "timezone": "Asia/Dubai",
            },
        }

    def __call__(self, city: str) -> ToolResponse:
        city_key = city.strip().lower()
        info = self._db.get(city_key)
        if not info:
            result = f'{{"city": "{city}", "error": "unknown city"}}'
        else:
            result = json.dumps({"city": city_key, **info})

        return ToolResponse(
            results=result,
            final_answer=False,  # ReAct can decide what to do next
            extras={},          # you can stash anything here into memory if you want
        )


calculator_tool = CalculatorTool()
city_info_tool = CityInfoTool()

toolkit = Toolkit(tools=[calculator_tool, city_info_tool])

INFO     | 2025-11-26 09:46:19 | toolkit.py:register_tool | Registered tool: calculator
INFO     | 2025-11-26 09:46:19 | toolkit.py:register_tool | Registered tool: city_info


In [160]:
from neurosurfer.agents.react.agent import ReActAgent, ReActConfig
from neurosurfer.models.chat_models.base import BaseChatModel


react_config = ReActConfig(
    skip_special_tokens=True,
    max_new_tokens=4096,
    temperature=0.7,
    log_internal_thoughts=True,
)

react_agent = ReActAgent(
    id="react_agent_main",
    llm=LLM,
    toolkit=toolkit,
    config=react_config,
    log_traces=True,
)

query = (
    "I'm deciding between visiting Paris or Tokyo. "
    "Use your tools to: "
    "1) look up the population of each city in millions, "
    "2) compute how many millions more people live in Tokyo than in Paris, "
    "3) then give me a final recommendation in natural language using that number."
)

res = react_agent.run(query=query, stream=False)


# # For streaming Response
# res = react_agent.run(query=query, stream=True)
# for chunk in res.response: 
#     # print(chunk, end="", flush=True)
#     pass

# traces
# trace_dict = res.traces.model_dump()

🧠 Thinking...

[react_agent_main] Tracing Start!
 ▶ [1][step.react_agent] agent_id='react_agent_main' label='react_agent.run'
     ▶ [2][step.llm.call] agent_id='react_agent_main' label='react_agent.loop.reason_llm_call'
        Thought: I need to look up the population of Paris and Tokyo first to compute the difference.
        
        Action: {
          "tool": "city_info",
          "inputs": {
            "city": "Paris"
          },
          "final_answer": false
        }        
        [🔧] Tool Selected: city_info
         ▶ [3][step.tool.execute] agent_id='react_agent_main' label='react_agent.loop.tool_execute'
            INFO: [🔧] Executing Tool: city_info, Attempt: 0, [📤] Inputs: {'city': 'Paris'}...
            INFO: [🔧] Tool Result: {"city": "paris", "population_millions": 2.1, "timezone": "Europe/Paris"}...
         ◀ [3][step.tool.execute] agent_id='react_agent_main' label='react_agent.loop.tool_execute' took 0.001s; error=False
        
     ◀ [2][step.llm.call] age

## RAG wiring so the agent “understands” the Neurosurf codebase

You’ll ingest the repo once, then run a retriever to answer code questions. The Planner can call the retriever first to form a precise implementation plan.

### FileReader and Chunker Test

In [64]:
# # scripts/index_repo_for_rag.py
# from pathlib import Path
# from neurosurfer.vectorstores.chroma import ChromaVectorStore
# from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder
# from neurosurfer.agents.rag.chunker import Chunker
# from neurosurfer.agents.rag.filereader import FileReader
# from neurosurfer.agents.rag import RAGAgent, RAGAgentConfig, RAGIngestorConfig

# chunker = Chunker()
# file_reader = FileReader()

# embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")

# rag_agent = RAGAgent(
#     llm=LLM,
#     embedder=embedder,
#     file_reader=file_reader,
#     chunker=chunker,
#     config=RAGAgentConfig(
#         top_k=5,
#         fixed_max_new_tokens=1024,
#         clear_collection_on_init=True
#     ),
#     ingestor_config=RAGIngestorConfig(
#         batch_size=64,
#         max_workers=4,
#         deduplicate=True,
#         normalize_embeddings=True,
#         default_metadata=None,
#         tmp_dir="./rag-storage",
#     ),
# )

# dir_path = "../neurosurfer"
# summary = rag_agent.ingest(sources=dir_path)

# summary

# retrival_results = rag_agent.retrieve(user_query="Explain how graph agent is initialized", top_k=10)

# print("max_new_tokens", retrival_results.max_new_tokens)
# print()
# print(retrival_results.context)


In [11]:
# # scripts/index_repo_for_rag.py
# from pathlib import Path
# from neurosurfer.rag.ingestor import RAGIngestor
# from neurosurfer.rag.chunker import Chunker
# from neurosurfer.rag.filereader import FileReader
# from neurosurfer.vectorstores.chroma import ChromaVectorStore
# from neurosurfer.models.embedders.sentence_transformer import SentenceTransformerEmbedder

# embedder = SentenceTransformerEmbedder("intfloat/e5-small-v2")
# vs = ChromaVectorStore(collection_name="neurosurf-repo")
# ing = RAGIngestor(
#     embedder=embedder,
#     vector_store=vs, 
#     chunker=Chunker(), 
#     file_reader=FileReader(),
#     default_metadata={"collection": "neurosurf"}
# )

# root_dir = Path(os.getcwd()).parent.joinpath("neurosurfer")
# ing.add_directory(root_dir)  # the repo root
# print(ing.build())


## Graph AGENT

### YAML Flow

In [92]:
# test web search tool
from neurosurfer.tools.websearch import WebSearchTool, WebSearchConfig
from neurosurfer.tools.toolkit import Toolkit

api_key = os.getenv("SERPAPI_KEY", "API Key not found...")
print("API Key: ", f"{api_key[:8]}...")

web_search_tool = WebSearchTool(
    config=WebSearchConfig(
        engine="serpapi",
        engine_kwargs={"api_key": api_key},
        max_results=3,
        enable_crawl=True,
        max_crawl_results=2,
        content_words_limit=2000,
        content_limit_strategy="distributive",
        summarize=False,
        top_k=10,
    ),
    llm=LLM,
)

# searches = web_search_tool(query="Importance of sleep in health.")
# print(searches)

toolkit = Toolkit(tools=[web_search_tool])
# print(toolkit.registry)

API Key:  f443633b...
WARNING  | 2025-11-18 14:44:24 | agent.py:__init__ | No vectorstore provided to RAGAgent, using default ChromaVectorStore. Initializing collection `neurosurfer-rag-agent`
[Init] ChromaVectorStore initialized with collection: neurosurfer-rag-agent
INFO     | 2025-11-18 14:44:35 | SentenceTransformer.py:__init__ | Use pytorch device_name: cuda:0
INFO     | 2025-11-18 14:44:35 | sentence_transformer.py:__init__ | SentenceTransformer embedding model initialized.
INFO     | 2025-11-18 14:44:35 | toolkit.py:register_tool | Registered tool: web_search


INFO     | 2025-11-18 14:44:40 | _common.py:_log_backoff | Backing off send_request(...) for 0.8s (requests.exceptions.ReadTimeout: HTTPSConnectionPool(host='us.i.posthog.com', port=443): Read timed out. (read timeout=15))


In [ ]:
# searches.results
# # print(searches.results["rag_content"])
# print(searches.results["llm_summary"])


{'query': 'Importance of sleep in health.',
 'summary': "Top 3 results out of ~988,000,000 results for: 'Importance of sleep in health.'\n1. How Sleep Works - Why Is Sleep Important? - NHLBI - NIH — https://www.nhlbi.nih.gov/health/sleep/why-sleep-important\n2. Better sleep: Why it's important for your health and tips to ... — https://health.ucdavis.edu/blog/cultivating-health/better-sleep-why-its-important-for-your-health-and-tips-to-sleep-soundly/2023/03\n3. Sleep is essential to health - Journal of Clinical Sleep Medicine — https://jcsm.aasm.org/doi/10.5664/jcsm.9476",
 'provider': 'serpapi',
 'elapsed_ms': 3004,
 'rag_content': 'Source: 355e311e80b4be26:cc35f490\nlth Sleep has become a precious commodity – we love it and need it, but rarely get enough of it. Busy schedules, kids, anxiety and technology can all get in the way of a good night sleep. Getting enough sleep can play an important role in your weight, emotional wellbeing, blood pressure, diabetes, mental and physical perfo

**GrpahAgent from YML file**

In [95]:
from neurosurfer.models.chat_models.base import BaseChatModel
from neurosurfer.agents.graph import GraphAgent, ManagerConfig

graph_agent = GraphAgent(
    llm=LLM,
    graph_yaml="blog_workflow.yml",
    toolkit=toolkit,
    manager_config=ManagerConfig(
        temperature=0.5,
        max_new_tokens=4096,
    ),
    manager_llm=LLM,
    log_traces=True
)

# Run workflow
graph_inputs = {
    "topic_title": "The Missing Middle Layer: Why LLM Systems Need Tool Routers, Not Bigger Models",
    "query": "Compose a 2000-2500 word blog on why tool-routing layers matter more than scaling LLM size, covering practical design patterns, examples, and tradeoffs.",
    "audience": "Intermediate ML engineers",
    "tone": "Practical and slightly opinionated",
}

results = graph_agent.run(inputs=graph_inputs)
# result = await run_async(executor.run(inputs=graph_inputs))


WARNING  | 2025-11-18 14:58:05 | utils.py:normalize_and_validate_graph_inputs | Ignoring extra inputs not declared in graph spec: audience, tone
INFO     | 2025-11-18 14:58:05 | toolkit.py:register_tool | Registered tool: web_search

[manager] Tracing Start!
 ▶ [1][step.llm.call] agent_id='manager' label='manager.compose_user_prompt'
 ◀ [1][step.llm.call] agent_id='manager' label='manager.compose_user_prompt' took 2.249s; error=False
[manager] Tracing End!

🧠 Thinking...

[research] Tracing Start!
 ▶ [1][step.agent] agent_id='research' label='agent.run'
     ▶ [2][step.llm.call] agent_id='research' label='agent.route_and_call.router_llm_call'
        INFO: Selected tool: web_search
        INFO: Raw inputs: {'query': 'The Missing Middle Layer: Why LLM Systems Need Tool Routers, Not Bigger Models'}
     ◀ [2][step.llm.call] agent_id='research' label='agent.route_and_call.router_llm_call' took 1.079s; error=False
     ▶ [3][step.tool.execute] agent_id='research' label='agent.route_and_ca

Batches: 100%|██████████| 1/1 [00:00<00:00, 53.60it/s]

INFO     | 2025-11-18 14:58:09 | agent.py:retrieve | [RAGRetriever] Retrieved 10 chunks
        INFO: Tool 'web_search' Tool Return: {'query': 'The Missing Middle Layer: Why LLM Systems Need Tool Routers, Not Bigger Models', 'summary...
     ◀ [3][step.tool.execute] agent_id='research' label='agent.route_and_call.tool_execute' took 0.933s; error=False
 ◀ [1][step.agent] agent_id='research' label='agent.run' took 2.014s; error=False
[research] Tracing End!


[manager] Tracing Start!
 ▶ [2][step.llm.call] agent_id='manager' label='manager.compose_user_prompt'


 ◀ [2][step.llm.call] agent_id='manager' label='manager.compose_user_prompt' took 0.790s; error=True
Error executing node outline: CUDA out of memory. Tried to allocate 158.00 MiB. 
GPU 0 has a total capacity of 11.61 GiB of which 120.00 MiB is free. Including 
non-PyTorch memory, this process has 10.70 GiB memory in use. Of the allocated 
memory 9.91 GiB is allocated by PyTorch, and 484.27 MiB is reserved by PyTorch 
but unallocated. If reserved but unallocated memory is large try setting 
PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See 
documentation for Memory Management  
(https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)
Returning partial results from executed nodes...
INFO     | 2025-11-18 14:58:10 | export.py:export_single_node | Exported node research output to exports/research_20251118_145807.json


In [11]:
# import json

# with open("graph_agent_results.json", "w") as writer:
#     json.dump(results.model_dump(), writer)

## Docs Gen Agent

In [161]:
from neurosurfer_labs.tools.doc_gen.dir_scanning import DirectoryScanTool
from neurosurfer_labs.tools.doc_gen.code_symbol_index import CodeSymbolIndexTool
from neurosurfer.tools.toolkit import Toolkit

root_dir = "../neurosurfer"
dir_scan_tool = DirectoryScanTool()
code_index_tool = CodeSymbolIndexTool()

toolkit = Toolkit(tools=[dir_scan_tool, code_index_tool])

params = {
    "project_root": root_dir,
    "docs_root": "docs",
}

scan_results = dir_scan_tool(project_root=root_dir)
scan_results.results

INFO     | 2025-11-26 14:42:37 | toolkit.py:register_tool | Registered tool: directory_scan
INFO     | 2025-11-26 14:42:37 | toolkit.py:register_tool | Registered tool: code_symbol_index
INFO     | 2025-11-26 14:42:37 | dir_scanning.py:__call__ | DirectoryScanTool scanning project_root=/home/nomi/workspace/neurosurfer/neurosurfer docs_root=None max_depth=None


{'project_root': '/home/nomi/workspace/neurosurfer/neurosurfer',
 'docs_root': None,
 'python_files': [{'path': 'version.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/version.py',
   'module': 'neurosurfer.version',
   'package': 'neurosurfer'},
  {'path': 'config.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/config.py',
   'module': 'neurosurfer.config',
   'package': 'neurosurfer'},
  {'path': 'diagnostics.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/diagnostics.py',
   'module': 'neurosurfer.diagnostics',
   'package': 'neurosurfer'},
  {'path': 'logger.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/logger.py',
   'module': 'neurosurfer.logger',
   'package': 'neurosurfer'},
  {'path': '__init__.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/__init__.py',
   'module': 'neurosurfer.__init__',
   'package': 'neurosurfer'},
  {'path': 'db/__init__.py',
   'abs_path': '/home/nomi/workspace

In [162]:
python_files = scan_results.results["python_files"]
module_paths = None
code_index_results = code_index_tool(python_files=python_files, module_paths=module_paths)
code_index_results.results


{'modules': {'neurosurfer.version': {'path': 'version.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/version.py',
   'package': 'neurosurfer',
   'classes': [],
   'functions': []},
  'neurosurfer.config': {'path': 'config.py',
   'abs_path': '/home/nomi/workspace/neurosurfer/neurosurfer/config.py',
   'package': 'neurosurfer',
   'classes': [{'name': 'BaseModelConfig',
     'docstring': 'Base configuration shared across all model types.\n\nAll model configs inherit these common parameters.\nCan be extended with model-specific settings.',
     'methods': [{'name': 'to_dict',
       'docstring': 'Convert config to dictionary for unpacking into model __init__.\n\nArgs:\n    exclude_none: Exclude fields with None values\n    \nReturns:\n    Dictionary of configuration parameters',
       'args': ['self', 'exclude_none']}]},
    {'name': 'AppConfig',
     'docstring': 'General application configuration.\n\nManages application-level settings like versioning, networking,\n

In [163]:
from neurosurfer.models.chat_models.base import BaseChatModel
from neurosurfer.agents.graph import GraphAgent, ManagerConfig

graph_agent = GraphAgent(
    llm=LLM,
    graph_yaml="docgen_workflow.yml",
    toolkit=toolkit,
    manager_config=ManagerConfig(
        temperature=0.5,
        max_new_tokens=4096,
    ),
    manager_llm=LLM,
    log_traces=True
)

# Run workflow
graph_inputs = {
    "project_root": "/home/nomi/workspace/neurosurfer/neurosurfer",
    "docs_root": "/home/nomi/workspace/neurosurfer/docs",
    "include_patterns": ["neurosurfer.agents.*"],
    "exclude_patterns": [],
    "sections": [],
    "doc_template": "",
    "mode": "structured",
}
print(graph_agent)
results = graph_agent.run(inputs=graph_inputs)
# result = await run_async(executor.run(inputs=graph_inputs))

INFO     | 2025-11-26 14:42:44 | toolkit.py:register_tool | Registered tool: directory_scan

[manager] Tracing Start!
 ▶ [1][step.llm.call] agent_id='manager' label='manager.compose_user_prompt'
 ◀ [1][step.llm.call] agent_id='manager' label='manager.compose_user_prompt' took 1.904s; error=False
[manager] Tracing End!

🧠 Thinking...

[scan_repo] Tracing Start!
 ▶ [1][step.agent] agent_id='scan_repo' label='agent.run'
     ▶ [2][step.llm.call] agent_id='scan_repo' label='agent.route_and_call.router_llm_call'
        INFO: Selected tool: directory_scan
        INFO: Raw inputs: {'include_patterns': ['**'], 'exclude_patterns': [], 'max_depth': 10}
     ◀ [2][step.llm.call] agent_id='scan_repo' label='agent.route_and_call.router_llm_call' took 0.987s; error=False
     ▶ [3][step.tool.execute] agent_id='scan_repo' label='agent.route_and_call.tool_execute'
INFO     | 2025-11-26 14:42:47 | dir_scanning.py:__call__ | DirectoryScanTool scanning project_root=/home/nomi/workspace/neurosurfer/neur

In [ ]:
# from dataclasses import dataclass, field
# from typing import Generator, Optional, Union, List

# @dataclass
# class TraceResult:
#     steps: List[str] = field(default_factory=list)
#     def add(self, msg: str):
#         self.steps.append(msg)

# class DummyTracer:
#     def __init__(self):
#         self.results = TraceResult()
#     def trace(self, msg: str):
#         self.results.add(msg)

# @dataclass
# class AgentResponse:
#     response: Union[str, Generator[str, None, None]]
#     traces: Optional[TraceResult] = None

# class DummyAgent:
#     def __init__(self):
#         self.tracer = DummyTracer()

#     def run(self) -> AgentResponse:
#         def generator():
#             for i in range(3):
#                 msg = f"token_{i}"
#                 self.tracer.trace(f"TRACE: {msg}")   # tracing inside generator
#                 yield msg
#         return AgentResponse(
#             response=generator(),
#             traces=self.tracer.results  # shared reference
#         )

# agent = DummyAgent()
# res = agent.run()

# print("=== BEFORE iterating generator ===")
# print(res.traces.steps)   # should be empty

# print("\n=== DURING streaming ===")
# for tok in res.response:
#     print("yielded:", tok)
#     print("current traces:", res.traces.steps)

# print("\n=== AFTER streaming ===")
# print(res.traces.steps)   # should have all traces


=== BEFORE iterating generator ===
[]

=== DURING streaming ===
yielded: token_0
current traces: ['TRACE: token_0']
yielded: token_1
current traces: ['TRACE: token_0', 'TRACE: token_1']
yielded: token_2
current traces: ['TRACE: token_0', 'TRACE: token_1', 'TRACE: token_2']

=== AFTER streaming ===
['TRACE: token_0', 'TRACE: token_1', 'TRACE: token_2']


In [2]:
# import json
# print(results.final['plan_docs'].replace('```json', '').replace('```', ''))

In [13]:
results.nodes['scan_repo'].traces.model_dump()
# results.nodes['scan_repo'].traces

{'steps': [{'step_id': 1,
   'kind': 'agent',
   'label': 'agent.run',
   'node_id': None,
   'agent_id': 'scan_repo',
   'started_at': 1764051228.8918066,
   'duration_ms': 840,
   'inputs': {'agent_type': 'Agent',
    'has_toolkit': True,
    'structured': False,
    'stream': False,
    'strict_tool_call': False},
   'outputs': {},
   'meta': {},
   'ok': True,
   'error': None,
   'logs': []},
  {'step_id': 2,
   'kind': 'llm.call',
   'label': 'agent.route_and_call.router_llm_call',
   'node_id': None,
   'agent_id': 'scan_repo',
   'started_at': 1764051228.8921106,
   'duration_ms': 834,
   'inputs': {'attempt': 1,
    'strict_tool_call': False,
    'system_prompt_len': 1959,
    'user_prompt_len': 342,
    'user_prompt': 'Scan the project_root and docs_root to build a structured index of packages, modules, and existing markdown docs. Include all Python files and markdown documents, and organize them into a compact JSON object with the following keys: "packages", "modules", "pyth

In [57]:
print(results.nodes['scan_repo'].traces.steps[1].inputs['user_prompt'])

Scan the project_root and docs_root to build a structured index of packages, modules, and existing markdown docs. Focus on Python files and documentation in the specified directories, using the include_patterns and exclude_patterns provided. Output a compact JSON object with the keys "packages", "modules", "python_files", and "doc_files".


In [4]:
from __future__ import annotations
from typing import List, Optional
from pydantic import BaseModel, Field, field_validator
from dataclasses import dataclass, field
from typing import Dict

@dataclass
class RouterRetryPolicy:
    """Retry tuning for routing + tool execution."""
    max_route_retries: int = 2
    max_tool_retries: int = 1
    backoff_sec: float = 0.7  # linear backoff

@dataclass
class AgentConfig:
    """
    Top-level configuration for the Agent.
    """
    # Routing:
    allow_input_pruning: bool = True    # drop unknown inputs not in ToolSpec
    repair_with_llm: bool = True        # ask LLM to repair invalid routing/inputs
    strict_tool_call: bool = False      # router must output JSON; else can answer in plain text
    # synonyms: Dict[str, Dict[str, str]] = field(default_factory=dict)  # field -> {from: to}

    # LLM defaults:
    temperature: float = 0.7
    max_new_tokens: int = 512
    return_stream_by_default: bool = False

    # Retries:
    retry: RouterRetryPolicy = field(default_factory=RouterRetryPolicy)

    # Structured-output options:
    strict_json: bool = True                  # enforce RFC 8259 JSON
    max_repair_attempts: int = 1              # for malformed JSON repairs



class NodeBudget(BaseChatModel):
    """
    Budget / LLM-related overrides per node.

    These map directly to AgentConfig fields:
        - temperature      -> AgentConfig.temperature
        - max_new_tokens   -> AgentConfig.max_new_tokens
        - return_stream_by_default -> AgentConfig.return_stream_by_default
    """

    max_new_tokens: Optional[int] = Field(
        default=None,
        description="Override AgentConfig.max_new_tokens for this node only.",
    )
    temperature: Optional[float] = Field(
        default=None,
        description="Override AgentConfig.temperature for this node only.",
    )
    return_stream_by_default: Optional[bool] = Field(
        default=None,
        description="Override AgentConfig.return_stream_by_default for this node only.",
    )


class NodePolicy(BaseChatModel):
    """
    Per-node policy that can override some AgentConfig settings and add
    node-level execution constraints (e.g., timeout).

    YAML example:

        nodes:
          - id: research
            policy:
              retries: 1
              timeout_s: 30
              budget:
                max_new_tokens: 180
                temperature: 0.2
              allow_input_pruning: false
              repair_with_llm: true
              strict_tool_call: true
    """

    retries: Optional[int] = Field(
        default=None,
        description="Override AgentConfig.retry.max_route_retries for this node.",
    )
    timeout_s: Optional[int] = Field(
        default=None,
        description=(
            "Soft timeout for this node in seconds. Execution isn't forcibly "
            "cancelled but the node will be marked as errored if exceeded."
        ),
    )
    budget: Optional[NodeBudget] = None

    # Direct AgentConfig-like overrides
    allow_input_pruning: Optional[bool] = None
    repair_with_llm: Optional[bool] = None
    strict_tool_call: Optional[bool] = None
    strict_json: Optional[bool] = None
    max_repair_attempts: Optional[int] = None

    class Config:
        extra = "ignore"  # ignore unknown keys under 'policy'

c = AgentConfig()

p = NodePolicy(budget=NodeBudget(temperature=1.2))

print(c)
print(p)


NameError: name 'BaseChatModel' is not defined

### Python API version (no YAML)

In [9]:
import asyncio
from graph import Graph, Node, NodePolicy, GraphConfig, GraphExecutor
from neurosurfer.tools import Toolkit
from neurosurfer.models.chat_models.openai import OpenAIModel

# Reuse your existing toolkit + model
llm = LLM  # already created in your environment
tk = toolkit

graph = Graph(
    name="calc_and_explain",
    config=GraphConfig(max_concurrency=2),
    inputs_schema={"prompt": str},
    nodes=[
        Node(
            id="rewrite",
            fn="general_query_assistant",  # adjust name if needed
            inputs={
                # swap "query" -> "prompt" if your tool expects "prompt"
                "query": (
                    "You will receive a user request. Extract a SINGLE pure arithmetic expression that can be "
                    "evaluated by a calculator (e.g., '(42 * 7) - 5^2' or '0.035 * 12000').\n"
                    "- Do NOT include explanations.\n"
                    "- Return ONLY the expression as plain text.\n\n"
                    "User request:\n${inputs.prompt}"
                )
            },
            outputs=["num1", "num2", "operation"],
            policy=NodePolicy(
                retries=1,
                timeout_s=30,
                budget={"max_new_tokens": 128, "temperature": 0.1},
            ),
        ),
        Node(
            id="compute",
            fn="calculator",
            inputs={"num1": "${rewrite.num1}", "num2": "${rewrite.num2}", "operation": "${rewrite.operation}"},
            outputs=["text"],
            policy=NodePolicy(retries=0, timeout_s=15),
        ),
        Node(
            id="explain",
            fn="general_query_assistant",
            inputs={
                "query": (
                    "Original request: ${inputs.prompt}\n"
                    "Calculator result: ${compute.text}\n\n"
                    "Write a brief, user-friendly explanation of the result (one short paragraph)."
                )
            },
            outputs=["text"],
            policy=NodePolicy(
                retries=1,
                timeout_s=30,
                budget={"max_new_tokens": 180, "temperature": 0.2},
            ),
        ),
    ],
    outputs={"answer": "${explain.text}"},
)

executor = GraphExecutor(llm=llm, toolkit=tk, max_concurrency=2)

result = await run_async(
    executor.run(graph, inputs={"prompt": "Compute 3.5% of 12000 and explain"}, stream=True)
)

print("OK:", result.ok)
print("Answer:\n", result.outputs["answer"])


OK: True
Answer:
 The calculator result for your request is ${compute.text}. This means that after performing the calculation based on your input, the final answer is ${compute.text}. Let me know if you need further assistance!


### Planner-based path (using the YAML as a skeleton)

In [ ]:
import asyncio, tempfile, pathlib
from graph import PlannerAgent, FlowLoader, GraphExecutor

# 1) Write the YAML to a temp file (only for this demo)
yaml_text = r"""
name: calc_and_explain
inputs:
  prompt: str
config:
  max_concurrency: 2
nodes:
  - id: rewrite
    kind: task
    fn: general_query_assistant
    inputs:
      query: |
        You will receive a user request. Extract a SINGLE pure arithmetic expression that can be
        evaluated by a calculator (e.g., "(42 * 7) - 5^2" or "0.035 * 12000").
        - Do NOT include explanations.
        - Return ONLY the expression as plain text.

        User request:
        ${inputs.prompt}
    outputs: ["text"]
    policy: { retries: 1, timeout_s: 30, budget: { max_new_tokens: 128, temperature: 0.1 } }

  - id: compute
    kind: task
    fn: calculator
    inputs: { expression: ${rewrite.text} }
    outputs: ["text"]

  - id: explain
    kind: task
    fn: general_query_assistant
    inputs:
      query: |
        Original request: ${inputs.prompt}
        Calculator result: ${compute.text}

        Write a brief, user-friendly explanation of the result (one short paragraph).
    outputs: ["text"]
    policy: { retries: 1, timeout_s: 30, budget: { max_new_tokens: 180, temperature: 0.2 } }

outputs: { answer: ${explain.text} }
""".strip()

tmp = pathlib.Path(tempfile.gettempdir()) / "calc_and_explain.yml"
tmp.write_text(yaml_text)

# 2) Use the planner with a skeleton (so it returns your YAML-based Graph)
planner = PlannerAgent(llm=LLM)  # LLM not used when skeleton is set
graph = planner.plan_from_query(query="Compute 3.5% of 12000 and explain", skeleton=str(tmp))

# 3) Execute
executor = GraphExecutor(llm=LLM, toolkit=toolkit, max_concurrency=2)
result = asyncio.run(executor.run(graph, inputs={"prompt": "Compute 3.5% of 12000 and explain"}))

print("OK:", result.ok)
print(result.outputs["answer"])


Test ToolsRouterAgent

In [8]:
query = "Perform the calculation 20 * 90"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")


INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Using tool: calculator
INFO     | 2025-11-06 11:08:32 | tools_router_agent.py:run | [router] Raw inputs: {'num1': 20.0, 'num2': 90.0, 'operation': 'multiply'}
1800.0

In [9]:
query = "Tell me a light-hearted joke!"

for chunk in tools_router_agent.run(query, temperature=0.7, max_new_tokens=4000):
    print(chunk, end="")

INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Using tool: general_query_assistant
INFO     | 2025-11-06 11:08:33 | tools_router_agent.py:run | [router] Raw inputs: {'query': 'Tell me a light-hearted joke!'}
Why don't skeletons fight each other? They don't have the guts!None

## ReactAgent

In [21]:
from neurosurfer.agents.react import ReActAgent, ReActConfig

react_agent = ReActAgent(
    toolkit=toolkit,
    llm=LLM,
    specific_instructions="Always be concise in your answers. Break the task into steps if needed.",
    config=ReActConfig(
        temperature=0.7,
        max_new_tokens=4096,
        allow_input_pruning=True,
        repair_with_llm=True,
        skip_special_tokens=True,
        verbose=True
    )
)

# print(react_agent._system_prompt())
TASK = """Calculate 300 - 300. Then tell me a light-hearted joke about that result."""

for chunk in react_agent.run(TASK):
    print(chunk, end="")




[🧠] Chain of Thoughts...
Thought: I will first calculate 300 - 300 using the calculator tool, and then I will use the general_query_assistant tool to find a light-hearted joke about the result.

Action: {
  "tool": "calculator",
  "inputs": {
    "num1": 300,
    "num2": 300,
    "operation": "subtract"
  },
  "final_answer": false
}

[🔧] Tool: calculator
[📤] Inputs: {'num1': 300, 'num2': 300, 'operation': 'subtract'}

Observation:



[🧠] Chain of Thoughts...
Thought: The result of the calculation is 0. Now, I will use the general_query_assistant tool to find a light-hearted joke about the result.

Action: {
  "tool": "general_query_assistant",
  "inputs": {
    "query": "Tell me a light-hearted joke about the number 0."
  },
  "final_answer": true
}

[🔧] Tool: general_query_assistant
[📤] Inputs: {'query': 'Tell me a light-hearted joke about the number 0.'}

Why did the number 0 break up with the number 8?  
Because it found someone more "8" (8) than a zero!

Observation: Why did the number 0 break up with the number 8?  
Because it found someone more "8" (8) than a zero!



[🧠] Chain of Thoughts...
Thought: The calculation result is 0, and the joke provided is ready. The final answer is complete.

Final Answer: The result of 300 - 300 is 0. Here's a light-hearted joke about it: Why did the number 0 break up with the number 8? Because it found someone more "8" (8) than a zero!